In [ ]:
# !pip install import_ipynb --quiet
# !git clone https://github.com/gmshroff/aml-public.git
# %cd aml-public

In [ ]:
import import_ipynb
import utils
import models
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pickle
import warnings
from ucimlrepo import fetch_ucirepo
warnings.filterwarnings('ignore') 
from matplotlib import pyplot as plt
from utils import MyDS

In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim

In [ ]:
import import_ipynb
import models
from models import Train,TrainVals

In [ ]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
Xf = iris.data.features 
yf = iris.data.targets 
classes=yf['class'].unique().tolist()

In [ ]:
perm = np.random.permutation(150).tolist()

In [ ]:
Xf,yf=Xf.iloc[perm],yf.iloc[perm]

In [ ]:
def encode_target(t,classes=classes):
    return classes.index(t)

In [ ]:
yf['label']=yf['class'].apply(encode_target)

In [ ]:
yf

In [ ]:
train_n=100

In [ ]:
X=torch.tensor(Xf.iloc[0:train_n].values, dtype=torch.float32)

In [ ]:
y=torch.tensor(yf.iloc[0:train_n]['label'].values,dtype=torch.float32)

In [ ]:
Xtest=torch.tensor(Xf.iloc[train_n:].values,dtype=torch.float32)
ytest=torch.tensor(yf[train_n:]['label'].values,dtype=torch.float32)

In [ ]:
def one_hot(y,n=3):
    oh=torch.zeros(y.shape[0],n)
    oh[torch.arange(y.shape[0]), y.long()]=1
    return oh

In [ ]:
y_train=one_hot(y)

In [ ]:
y_train

In [ ]:
class DotSoft(nn.Module):
    def __init__(self,x_train,y_train):
        super(DotSoft,self).__init__()
        self.x_train=x_train
        self.y_train=y_train
        self.S=nn.Softmax(dim=-1)
    def __call__(self,X):
        XN=X/torch.sqrt(torch.sum(X*X,1).unsqueeze(-1))
        xN=self.x_train/torch.sqrt(torch.sum(self.x_train*self.x_train,1).unsqueeze(-1))
        # print(self.S(XN@xN.transpose(1,0)))        
        return self.S(XN@xN.transpose(1,0))@self.y_train


In [ ]:
dS=DotSoft(X,y_train)

In [ ]:
ytest

In [ ]:
models.accuracy(dS,X,y)

In [ ]:
models.accuracy(dS,Xtest,ytest)

In [ ]:
class SoftNN(nn.Module):
    def __init__(self,x_train,y_train,d=10,lr=1e-3,weight_decay=0.001):
        super(SoftNN,self).__init__()
        self.x_train=x_train
        self.y_train=y_train
        self.W=torch.rand(x_train.shape[1],d,requires_grad=True)
        self.b=torch.rand(1,1,requires_grad=True)
        self.S=nn.Softmax(dim=-1)
        self.optimizer = optim.Adam([self.W,self.b],lr=lr,weight_decay=weight_decay)
        self.relu=nn.Tanh()
    def __call__(self,X):
        Z=self.relu(self.x_train@self.W+self.b)
        V=self.relu(X@self.W+self.b)
        ZN=Z/torch.sqrt(torch.sum(Z*Z,1).unsqueeze(-1))
        VN=V/torch.sqrt(torch.sum(V*V,1).unsqueeze(-1))
        # print(self.S(VN@ZN.transpose(1,0)))
        return self.S(VN@ZN.transpose(1,0))@self.y_train


In [ ]:
class SoftNNAttention(nn.Module):
    def __init__(self,x_train,y_train,d=10,lr=1e-3,weight_decay=0.001):
        super(SoftNNAttention,self).__init__()
        self.x_train=x_train
        self.y_train=y_train
        self.W=torch.rand(x_train.shape[1],d,requires_grad=True)
        self.U=torch.rand(x_train.shape[1],d,requires_grad=True)
        self.b=torch.rand(1,1,requires_grad=True)
        self.c=torch.rand(1,1,requires_grad=True)
        self.S=nn.Softmax(dim=-1)
        self.optimizer = optim.Adam([self.W,self.U,self.b,self.c],lr=lr,weight_decay=weight_decay)
        self.relu=nn.Tanh()
    def __call__(self,X):
        Z=self.relu(self.x_train@self.W+self.b)
        V=self.relu(X@self.U+self.c)
        ZN=Z/torch.sqrt(torch.sum(Z*Z,1).unsqueeze(-1))
        VN=V/torch.sqrt(torch.sum(V*V,1).unsqueeze(-1))
        # print(self.S(VN@ZN.transpose(1,0)))
        return self.S(VN@ZN.transpose(1,0))@self.y_train


In [ ]:
snn=SoftNN(X,y_train)

In [ ]:
snn=SoftNNAttention(X,y_train)

In [ ]:
snn,losses,accs=models.TrainVals(snn,X.numpy(),y,epochs=100,verbose=True)

In [ ]:
models.accuracy(snn,X,y)

In [ ]:
models.accuracy(snn,Xtest,ytest)

In [ ]:
train_ds=MyDS(Xf.iloc[0:train_n].values,yf[0:train_n]['label'].values)

In [ ]:
test_ds=MyDS(Xf.iloc[train_n:].values,yf[train_n:]['label'].values)

In [ ]:
dloader=torch.utils.data.DataLoader(train_ds,batch_size=32,shuffle=False)

In [ ]:
snn,losses,accs=Train(snn,dloader,epochs=100,verbose=True,val_ds=test_ds,plot_accs=True)

In [ ]:
models.accuracy(snn,X,y)

In [ ]:
models.accuracy(snn,Xtest,ytest)

In [ ]:
snn.W,snn.U,snn.b,snn.c